

Topics Covered

* Protocols of the python language
* Reducing boilerplate with data classes
* Subclassing built-in types
* Accessing methods from superclasses
* Slots


### Reducing boilerplate with data classes

`dataclass` module in py provides a decorator and function that allows you to easily add generated special methods to your own class. 

In [ ]:
%load_ext autoreload
%autoreload 2

In [3]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        """Add two vectors using + operator"""
        return Vector(
            self.x + other.x,
            self.y + other.y,
        )

    def __sub__(self, other):
        """Subtract two vectors using - operator"""
        return Vector(
            self.x - other.x,
            self.y - other.y,
        )

    def __repr__(self):
        """Return textual representation of vector"""
        return f"<Vector: x={self.x}, y={self.y}>"

    def __eq__(self, other):
        """Compare two vectors for equality"""
        return self.x == other.x and self.y == other.y

Below are the results of using the class based on usage with common operators

In [6]:
print(Vector(2,3))
print(Vector(5,3) + Vector(1,2))
print(Vector(5,3) - Vector(1,2))
print(Vector(1,1) == Vector(2,2))
print(Vector(2,2) == Vector(2,2))

<Vector: x=2, y=3>
<Vector: x=6, y=5>
<Vector: x=4, y=1>
False
True


The Vector classes above is very repetivive that can be avoited using the `dataclass` module, like so:

In [7]:
from dataclasses import dataclass


@dataclass
class Vector:
    x: int
    y: int

    def __add__(self, other):
        """Add two vectors using + operator"""
        return Vector(
            self.x + other.x,
            self.y + other.y,
        )

    def __sub__(self, other):
        """Subtract two vectors using - operator"""
        return Vector(
            self.x - other.x,
            self.y - other.y,
        )

In [8]:
print(Vector(2,3))
print(Vector(5,3) + Vector(1,2))
print(Vector(5,3) - Vector(1,2))
print(Vector(1,1) == Vector(2,2))
print(Vector(2,2) == Vector(2,2))

Vector(x=2, y=3)
Vector(x=6, y=5)
Vector(x=4, y=1)
False
True


Above, with the addition of the `@dataclass` decorator, the `@dataclass` reads annotations of the `Vector` class attribute and automatically creates the `__init__()`, `__repr__()` and `__eq__()` methods and the defual equality coparision assums that the 2 instances ar eequal if all their respective attributes are equal to eahc other. 

Additionally, by adding in the `frozen=True` arugment the `@dataclass` decorator makes the __class instance immutable__ _(cannot modify any of the attributes)_, and therefore, we can use them as dictionary keys and content sets. 

In [23]:
@dataclass(frozen=True)
class FrozenVector:
    x: int
    y: int

In [24]:
k = FrozenVector(2,5)
print(k)
k.x = 10

FrozenVector(x=2, y=5)


FrozenInstanceError: cannot assign to field 'x'

In [25]:
k1 = Vector(2,3)
print(k1)
k1.x = 4
print(k1)

Vector(x=2, y=3)
Vector(x=4, y=3)


---
## MRO and Superclass

`super()` is builtin funciton that returns a proxy object _(temporary object of the superclass)_ that allows us to access mehtods of the base class. This enables us to:
1. avoid using the base class name explicitly
2. working with multiple inheritances


```
>>> super
<class 'super'>
>>> isinstance(super, type)
```

Below is the non-superclass inheritance, where we are accessing a class attribute/method by calling the parent class directly and passing `self` as the first argument. like so:

In [47]:
class Mama:  # this is the old way 
    def says(self): 
        print('do your homework') 
 
          
class Sister(Mama): 
    def says(self): 
        Mama.says(self) 
        print('and clean your bedroom')

In [56]:
print(Mama().says())
print("====")
print(Sister().says())

do your homework
None
====
do your homework
and clean your bedroom
None


In the `Sister` class we call the `Mama.says(self)` is the usage of the __parent class__ indicating the funciton `says()` belongs to Mama will be called... however, the instance which its being called is provided as the `self` argument that is the instance of the `Sister` class. This is the perfect scenario we can use the `super()`, 

like so:

In [57]:
class Sister(Mama):
    def says(self):
        #super(Sister, self).says()
        super().says()
        print('and clean your bedroom')

In [58]:
Sister().says()

do your homework
and clean your bedroom


We do this and dont even pass any arguments with the addition of `super()`. In face we can use `usper` anywhere the _explicit call to the method of superclass implinentation is required_. 

In [88]:
class Pizza: 
    def __init__(self, toppings): 
        self.toppings = toppings 
 
    def __repr__(self): 
        return "Pizza with " + " and ".join(self.toppings) 
 
    @classmethod 
    def recommend(cls): 
        """Recommend some pizza with arbitrary toppings,""" 
        return cls(['spam', 'ham', 'eggs']) 
 
 
class VikingPizza(Pizza): 
    @classmethod 
    def recommend(cls): 
        """Use same recommendation as super but add extra spam""" 
        recommended = super(VikingPizza).recommend() 
        recommended.toppings += ['spam'] * 5 
        return recommended 

In [83]:
VikingPizza('ham').recommend

<bound method VikingPizza.recommend of <class '__main__.VikingPizza'>>

In [76]:
VikingPizza('spam')

Pizza with s and p and a and m

In [80]:
VikingPizza('spam').recommend

<bound method VikingPizza.recommend of Pizza with s and p and a and m>

In [89]:
VikingPizza('spam').recommend

<bound method VikingPizza.recommend of <class '__main__.VikingPizza'>>

---
### Method Resolution Order (MRO)

Since we are going to have class inheritance, the class that is being heherited is called the __Parent class__ or __Superclass__, while the class that inherits is called hte __Child class__ or __Subclass__.

__Method Resolution Order (MRO)__  defines the __order__ in which the _base classes_ are searced when executing a method. Generally, the method is searched _within a class_ and the it follows the _order we specified while inheriting_


In [98]:
class CommonBase: 
    def method(self): 
        print('CommonBase') 
 
 
class Base1(CommonBase): 
    pass 
 
 
class Base2(CommonBase): 
    def method(self): 
        print('Base2') 
 
 
class MyClass(Base1, Base2): 
    pass

Above `Base` and `Base2` inherit form the Parent Class `CommonBase`

In [99]:
MyClass().method()

Base2


---
## Advanced attribute access patterns

__name mangling__ - every time an attribute is prefixed by __, it is renamed by the interpreter on the fly:


- _https://www.geeksforgeeks.org/name-mangling-in-python/_
- _https://www.geeksforgeeks.org/private-variables-python/_

In [125]:
class MyClass: 
    __secret_value = 1 
    _secret_value = 1 
    secret_value = 1

In [126]:
print(MyClass()._secret_value)
print(MyClass().secret_value)
print(MyClass().__secret_value)

1
1


AttributeError: 'MyClass' object has no attribute '__secret_value'

This feature could be used to protect the access of some attributes, but, in practice, __ should never be used. When an attribute is not public, the convention to use is a _ prefix. This does not invoke any name mangling algorithm, but just documents the attribute as a private element of the class and is the prevailing style.



### Descriptors

## Descriptors

__Descriptors__ lets you customize what should be done when you refer to an attribute of an object. They are classes that define how attributes of another class can be accesses. In other words, a class can delegate the management of an attribute to another one. 

- _https://realpython.com/python-descriptors/_
- _https://www.python-course.eu/python3_descriptors.php_

The descriptor classes are based on three special methods that form the __dscriptor protocol__:
* `__set__(self, obj, value)`: This is called whenever the attribute is set. In the following examples, I will refer to this as a setter.
* `__get__(self, obj, owner=None)`: This is called whenever the attribute is read (referred to as a getter).
* `__delete__(self, obj)`: This is called when del is invoked on the attribute


In [1]:
class Verbose_attribute():
    def __get__(self, obj, type=None) -> object:
        print("accessing the attribute to get the value")
        return 42
    def __set__(self, obj, value) -> None:
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")

class Foo():
    attribute1 = Verbose_attribute()

my_foo_object = Foo()
x = my_foo_object.attribute1
print(x)

accessing the attribute to get the value
42


In [2]:
class RevealAccess(object): 
    """A data descriptor that sets and returns values 
       normally and prints a message logging their access. 
    """ 
 
    def __init__(self, initval=None, name='var'): 
        self.val = initval 
        self.name = name 
 
    def __get__(self, obj, objtype): 
        print('Retrieving', self.name) 
        return self.val 
 
    def __set__(self, obj, val): 
        print('Updating', self.name) 
        self.val = val 
 
 
class MyClass(object): 
    x = RevealAccess(10, 'var "x"') 
    y = 5

The preceding example clearly shows that, if a class has the data descriptor for the given attribute, then the descriptor's` __get__()` method is called to return the value every time the instance attribute is retrieved

In [3]:
m = MyClass()
m.x

Retrieving var "x"


10

and __set__() is called whenever a value is assigned to such an attribute

In [4]:
m.x = 20

Updating var "x"


In [5]:
m.x

Retrieving var "x"


20

In [6]:
m.y

5

### Properties

__Properties__ provide a built-in _descriptor_ type that knows how to __link an attribute to a set of methods__. `property` takes 4 optional arguments: `fget, fset, fdel` and,  `doc`

_https://realpython.com/python-descriptors/_

In [17]:
class Rectangle:
    def __init__(self, x1, y1, x2, y2):
        self.x1, self.y1 = x1, y1
        self.x2, self.y2 = x2, y2

    @property
    def width(self):
        """rectangle width measured from left"""
        return self.x2 - self.x1

    @width.setter
    def width(self, value):
        self.x2 = self.x1 + value

    @property
    def height(self):
        """rectangle height measured from top"""
        return self.y2 - self.y1

    @height.setter
    def height(self, value):
        self.y2 = self.y1 + value

In [27]:
rec = Rectangle(10, 10, 25, 34)
rec.width, rec.height

(15, 24)

In [31]:
rec.width = 100
rec.__dict__

{'x1': 10, 'y1': 10, 'x2': 110, 'y2': 34}